In [30]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/ECE570/FinalProject/RegluarLRNN-master')
!ls

import torch
import torch.nn as nn
import math

import torch
import torch.nn as nn
import argparse

# !pip install torch --upgrade
# from torch._dynamo.utils import CompileProfiler
# prof = CompileProfiler()
from torch.utils.data import DataLoader, TensorDataset
torch.set_float32_matmul_precision('high')

from google.colab import files
import csv
#files.download('/content/arith_sequences.csv')

import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import torch.nn.functional as F
import math

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Arithmetic.ipynb     cycle.csv	  cycle.py		pair.csv    __pycache__
arith_sequences.csv  Cycle.ipynb  DataGeneration.ipynb	Pair.ipynb  README.md


In [43]:
#ARITHEMETIC DATASET
def arith_task_single(M, seq_len):
    if seq_len % 2 == 0:
        print('seq_len must be odd, reduce seq_len by 1')
        seq_len -= 1

    # (seq_len+1)//2 # of numbers for the sequence
    # (seq_len-1)//2 # of symbols for the sequence
    # (seq_len+1)//2 # of answers for the sequence
    # M, M+1, M+2 correspond to +, -, *

    nums = torch.randint(0, M, ((seq_len + 1) // 2,))
    symb = torch.randint(M, M + 3, ((seq_len - 1) // 2,))
    ans = torch.zeros((seq_len + 1) // 2, dtype=torch.int64)
    seqs = torch.zeros(seq_len, dtype=torch.int64)

    # Initialize the first number and answer
    ans[0] = nums[0]
    seqs[0] = nums[0]

    l, r = 0, nums[0]
    for i in range((seq_len - 1) // 2):
        if symb[i] == M:
            l, r = l + r, nums[i + 1]
        elif symb[i] == M + 1:
            l, r = l + r, -nums[i + 1]
        else:
            r = (r * nums[i + 1]) % M
        ans[i + 1] = (l + r) % M
        seqs[2 * i + 1] = symb[i]
        seqs[2 * i + 2] = nums[i + 1]

    return seqs.tolist(), ans.tolist()

def generate_and_save_data(M, batch_size, seq_len, num_batches, file_path):

    # Save the sequences and answers to a CSV file
    with open(file_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)

        #Generate data for each batch and write to CSV
        print("NUMBER OF DATA ENTRIES TOTAL:" + str(num_batches * batch_size))
        for i in range(num_batches * batch_size):
            if i%100000==0:
                print(f'Entry Generated: {i}')
            seqs, ans = arith_task_single(M, seq_len)
            csvwriter.writerow(seqs)
            csvwriter.writerow(ans)


# Example usage
M = 5
seq_len = 39  # Ensure seq_len is odd

# Generate one data entry (sequence and answer)
seqs, ans = arith_task_single(M, seq_len)

print("Sequence:", seqs)
print("Answer:", ans)

batch_size = 128
num_batches = 400
generate_and_save_data(M, batch_size, seq_len, num_batches, '/content/drive/MyDrive/ECE570/FinalProject/RegluarLRNN-master/arith_sequences.csv')

Sequence: [2, 5, 4, 6, 0, 7, 3, 5, 2, 5, 0, 5, 2, 5, 1, 6, 1, 6, 1, 6, 0, 7, 1, 5, 4, 5, 0, 6, 1, 7, 4, 5, 4, 6, 1, 5, 4, 7, 3]
Answer: [2, 1, 1, 1, 3, 3, 0, 1, 0, 4, 4, 4, 3, 3, 2, 4, 3, 2, 1, 4]
NUMBER OF DATA ENTRIES TOTAL:51200
Entry Generated: 0


In [42]:
#CYCLE SUM DATASET
def cycle_task(M=2, seq_len=10):
    # Generate a 1D tensor (vector) of random tokens
    sequence = torch.randint(0, M, (seq_len,))  # Shape: (seq_len)
    # Apply modulo operation to wrap values
    answers = sequence.cumsum(dim=-1) % M

    # Convert tensors to lists for output
    return sequence.tolist(), answers.tolist()


def cycle_task_original(M=3, bs=10, seq_len=10):
    tokens = torch.randint(0,M, (bs,seq_len))
    cycle = tokens.cumsum(dim=-1) % M
    return tokens, cycle

def generate_and_save_data(M, batch_size, seq_len, num_batches, file_path):

    # Save the sequences and answers to a CSV file
    with open(file_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)

        #Generate data for each batch and write to CSV
        print("NUMBER OF DATA ENTRIES TOTAL:" + str(num_batches * batch_size))
        for i in range(num_batches * batch_size):
            if i%100000==0:
                print(f'Entry Generated: {i}')
            seqs, ans = cycle_task(M, seq_len)
            csvwriter.writerow(seqs)
            csvwriter.writerow(ans)


# Example usage
M = 5
seq_len = 40  # Ensure seq_len is even

# Generate one data entry (sequence and answer)
seqs, ans = cycle_task(M, seq_len)
seqs_old, ans_old = cycle_task_original(M, 1, seq_len)

print("Answers:", ans)
print("Answers old:", ans_old)
print("Sequence:", seqs)
print("Sequence old:", seqs_old)

print("Answers:", len(ans))
print("Answers old:", len(ans_old.tolist()))
print("Sequence:", len(seqs))
print("Sequence old:", len(seqs_old.tolist()))

batch_size = 128
num_batches = 400
generate_and_save_data(M, batch_size, seq_len, num_batches, '/content/drive/MyDrive/ECE570/FinalProject/RegluarLRNN-master/cycle.csv')

Answers: [2, 0, 1, 3, 1, 2, 4, 2, 3, 2, 1, 2, 2, 3, 2, 4, 3, 1, 1, 4, 4, 4, 2, 2, 3, 2, 4, 1, 2, 0, 2, 0, 3, 3, 0, 1, 0, 0, 3, 0]
Answers old: tensor([[2, 3, 2, 2, 4, 3, 4, 0, 1, 4, 4, 2, 2, 2, 1, 2, 4, 2, 4, 2, 3, 2, 1, 4,
         3, 1, 3, 3, 4, 4, 1, 3, 3, 1, 1, 2, 2, 2, 1, 2]])
Sequence: [2, 3, 1, 2, 3, 1, 2, 3, 1, 4, 4, 1, 0, 1, 4, 2, 4, 3, 0, 3, 0, 0, 3, 0, 1, 4, 2, 2, 1, 3, 2, 3, 3, 0, 2, 1, 4, 0, 3, 2]
Sequence old: tensor([[2, 1, 4, 0, 2, 4, 1, 1, 1, 3, 0, 3, 0, 0, 4, 1, 2, 3, 2, 3, 1, 4, 4, 3,
         4, 3, 2, 0, 1, 0, 2, 2, 0, 3, 0, 1, 0, 0, 4, 1]])
Answers: 40
Answers old: 1
Sequence: 40
Sequence old: 1
NUMBER OF DATA ENTRIES TOTAL:51200
Entry Generated: 0


In [41]:
#PAIR DATASET
def Pair(M=2, seq_len=10):
    # Generate a 1D tensor (vector) of random tokens
    sequence = torch.randint(0, M, (seq_len,))  # Shape: (seq_len)
    # Compare the first token with all other tokens
    answers = (sequence[0] == sequence).to(torch.int64)

    # Convert tensors to lists for output
    return sequence.tolist(), answers.tolist()

def pair_taskOLD(M=2, bs=10, seq_len=10):
    tokens = torch.randint(0,M, (bs,seq_len))
    pair = (tokens[:,0:1] == tokens).to(torch.int64)
    return tokens, pair


def generate_and_save_data(M, batch_size, seq_len, num_batches, file_path):

    # Save the sequences and answers to a CSV file
    with open(file_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)

        #Generate data for each batch and write to CSV
        print("NUMBER OF DATA ENTRIES TOTAL:" + str(num_batches * batch_size))
        for i in range(num_batches * batch_size):
            if i%100000==0:
                print(f'Entry Generated: {i}')
            seqs, ans = Pair(M, seq_len)
            csvwriter.writerow(seqs)
            csvwriter.writerow(ans)


# Example usage
M = 5
seq_len = 10  # Ensure seq_len is even

seqs, ans = Pair(M, seq_len)
seqs_old, ans_old = pair_taskOLD(M, 1, seq_len)

print("Answers:", ans)
print("Answers old:", ans_old)
print("Sequence:", seqs)
print("Sequence old:", seqs_old)

print("Answers:", len(ans))
print("Answers old:", len(ans_old.flatten().tolist()))
print("Sequence:", len(seqs))
print("Sequence old:", len(seqs_old.flatten().tolist()))

batch_size = 128
num_batches = 400
generate_and_save_data(M, batch_size, seq_len, num_batches, '/content/drive/MyDrive/ECE570/FinalProject/RegluarLRNN-master/pair.csv')

Answers: [1, 0, 0, 0, 0, 1, 0, 1, 0, 0]
Answers old: tensor([[1, 0, 1, 1, 1, 1, 0, 0, 0, 0]])
Sequence: [4, 3, 2, 0, 3, 4, 3, 4, 0, 0]
Sequence old: tensor([[1, 4, 1, 1, 1, 1, 4, 2, 2, 3]])
Answers: 10
Answers old: 10
Sequence: 10
Sequence old: 10
NUMBER OF DATA ENTRIES TOTAL:51200
Entry Generated: 0


In [ ]:
from google.colab import files
import csv
#files.download('/content/arith_sequences.csv')